In [ ]:
# Importing required libraries
import os
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dropout, Input, Concatenate, Conv2D, MaxPooling2D, Conv2DTranspose, UpSampling2D, BatchNormalization
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy, SparseCategoricalCrossentropy
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import glob

# 2D U-Net model
def unet(input_shape, num_classes):
    inputs = Input(input_shape)

    # Contracting path
    conv1 = Conv2D(32, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, 3, activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, 3, activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, 3, activation='relu', padding='same')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(512, 3, activation='relu', padding='same')(pool4)
    conv5 = Conv2D(512, 3, activation='relu', padding='same')(conv5)
    drop5 = Dropout(0.5)(conv5)
    pool5 = MaxPooling2D(pool_size=(2, 2))(drop5)

    conv6 = Conv2D(1024, 3, activation='relu', padding='same')(pool5)
    conv6 = Conv2D(1024, 3, activation='relu', padding='same')(conv6)
    drop6 = Dropout(0.5)(conv6)

    # Expansive path
    up7 = Conv2DTranspose(512, 2, strides=(2, 2), padding='same')(drop6)
    up7 = Concatenate(axis=3)([up7, drop5])
    conv7 = Conv2D(512, 3, activation='relu', padding='same')(up7)
    conv7 = Conv2D(512, 3, activation='relu', padding='same')(conv7)

    up8 = Conv2DTranspose(256, 2, strides=(2, 2), padding='same')(conv7)
    up8 = Concatenate(axis=3)([up8, drop4])
    conv8 = Conv2D(256, 3, activation='relu', padding='same')(up8)
    conv8 = Conv2D(256, 3, activation='relu', padding='same')(conv8)

    up9 = Conv2DTranspose(128, 2, strides=(2, 2), padding='same')(conv8)
    up9 = Concatenate(axis=3)([up9, conv3])
    conv9 = Conv2D(128, 3, activation='relu', padding='same')(up9)
    conv9 = Conv2D(128, 3, activation='relu', padding='same')(conv9)

    up10 = Conv2DTranspose(64, 2, strides=(2, 2), padding='same')(conv9)
    up10 = Concatenate(axis=3)([up10, conv2])
    conv10 = Conv2D(64, 3, activation='relu', padding='same')(up10)
    conv10 = Conv2D(64, 3, activation='relu', padding='same')(conv10)

    up11 = Conv2DTranspose(32, 2, strides=(2, 2), padding='same')(conv10)
    up11 = Concatenate(axis=3)([up11, conv1])
    conv11 = Conv2D(32, 3, activation='relu', padding='same')(up11)
    conv11 = Conv2D(32, 3, activation='relu', padding='same')(conv11)

    # Output layer
    conv12 = Conv2D(num_classes, 1, activation='softmax')(conv11)
    model = Model(inputs=inputs, outputs=conv12)
    return model

# Load the training dataset
image_folder_train = 'path/train/dataset'
mask_folder_train = 'path/train/mask/dataset'

data_train = []
label_train = []
for image_path in sorted(glob.glob('{}/*'.format(image_folder_train))):
    img = Image.open(image_path)
    img = img.resize((256, 256))
    img = np.array(img)
    a = np.array(img)
    data_train.append(a)
for image_path in sorted(glob.glob('{}/*'.format(mask_folder_train))):
    img = Image.open(image_path)
    img = img.resize((256, 256))
    img = np.array(img)
    a = np.array(img, dtype='uint8')
    label_train.append(a)

# Show the training dataset with mask
for i in range(len(data_train)):
  plt.imshow(data_train[i])
  plt.axis('off')
  plt.show()
  plt.imshow(label_train[i])
  plt.axis('off')
  plt.show()

# Load the testing dataset
image_folder_test = '/path/Test/dataset'
mask_folder_test = '/path/Test/mask/dataset'

data_test = []
label_test = []
for image_path in sorted(glob.glob('{}/*'.format(image_folder_test))):
    img = Image.open(image_path)
    img = img.resize((256, 256))
    img = np.array(img)
    a = np.array(img)
    data_test.append(a)
for image_path in sorted(glob.glob('{}/*'.format(mask_folder_test))):
    img = Image.open(image_path)
    img = img.resize((256, 256))
    img = np.array(img)
    a = np.array(img, dtype='uint8')
    label_test.append(a)

# Show the testing dataset with mask
for i in range(len(data_test)):
  plt.imshow(data_test[i])
  plt.axis('off')
  plt.show()
  plt.imshow(label_test[i])
  plt.axis('off')
  plt.show()

# Normelization of data
data_train = np.array(data_train)
label_train = np.array(label_train)
data_train = data_train / 255.0

data_test = np.array(data_test)
label_test = np.array(label_test)
data_test = data_test / 255.0


# Replace with your actual data
X_train = np.array(data_train).reshape(-1, 256, 256, 3)
X_test = np.array(data_test).reshape(-1, 256, 256, 3)

y_train = np.array(label_train).reshape(-1, 256, 256, 1)
y_test = np.array(label_test).reshape(-1, 256, 256, 1)

# compile the model and fit the model
input_shape = (256, 256, 3)
model = unet(input_shape, num_classes=2)
model.compile(optimizer=Adam(), loss=SparseCategoricalCrossentropy(), metrics=['accuracy'])
batch_size = 1
epochs = 200
history=model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test))

# Get training and test loss
train_loss = history.history['loss']
test_loss = history.history['val_loss']

# Get training and test accuracy
train_acc = history.history['accuracy']
test_acc = history.history['val_accuracy']

# Plot loss
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(train_loss, label='Training Loss')
plt.plot(test_loss, label='Test Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Test Loss')
plt.legend()

# Plot accuracy
plt.subplot(1, 2, 2)
plt.plot(train_acc, label='Training Accuracy')
plt.plot(test_acc, label='Test Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Test Accuracy')
plt.legend()

plt.show()

# test and prediction
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=-1)
y_true_classes = np.squeeze(y_test, axis=-1)

# find the performance parameter from confussion matrix
conf_matrix = confusion_matrix(y_true_classes.flatten(), y_pred_classes.flatten())
TP = conf_matrix[1, 1]
TN = conf_matrix[0, 0]
FP = conf_matrix[0, 1]
FN = conf_matrix[1, 0]

# Sensitivity (Recall)
sensitivity = TP / (TP + FN)

# Specificity
specificity = TN / (TN + FP)

# Precision
precision = TP / (TP + FP)

# Dice Coefficient (F1 Score)
dice_coefficient = 2 * TP / (2 * TP + FP + FN)

# Accuracy
accuracy = (TP + TN) / (TP + TN + FP + FN)

# IoU(intersection over union)or jaccard index
iou = TP / (TP + FP + FN)

print("Confusion Matrix:")
print(conf_matrix)
print()
print("Sensitivity (Recall)                        :", sensitivity)
print()
print("Specificity (True Negative Rate)            :", specificity)
print()
print("Precision                                   :", precision)
print()
print("Dice Coefficient (F1 Score)                 :", dice_coefficient)
print()
print("Accuracy                                    :", accuracy)
print()
print("IoU(intersection over union)or jaccard index:", iou)

# visualization of predicted mask with graundtruth
predictions =1-model.predict(data_test)
sample_index = 2
image = np.squeeze(data_test[sample_index])
plt.subplot(1, 3, 1)
plt.imshow(image)
plt.title('Original Image')
plt.subplot(1, 3, 2)
plt.imshow(label_test[sample_index], cmap='gray')
plt.title('Ground Truth Mask')
plt.subplot(1, 3, 3)
plt.imshow(predictions[sample_index,:, :, 0] , cmap='gray')
plt.title('Predicted Mask')
plt.show()

# total case visualization of predicted mask with graundtruth
predictions =1-model.predict(data_test)
for i in range(len(data_test)):
  sample_index = i
  image = np.squeeze(data_test[sample_index])
  plt.subplot(1, 3, 1)
  plt.imshow(image)
  plt.title('Original Image')
  plt.subplot(1, 3, 2)
  plt.imshow(label_test[sample_index], cmap='gray')
  plt.title('Ground Truth Mask')
  threshold = 0.5
  predicted_mask = predictions[sample_index, :, :, 0] > threshold
  plt.subplot(1, 3, 3)
  plt.imshow(predictions[sample_index,:, :, 0] , cmap='gray')
  plt.title('Predicted Mask')
  plt.show()